In [ ]:
# Spark ML : Titanic

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName('Spark DataFrames') \
    .getOrCreate()
sc = spark.sparkContext

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

In [2]:
titanic = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load("/demo/titanic/titanic-train.csv"))

In [ ]:
display(titanic)

In [ ]:
titanic.show(10)

In [ ]:
titanic.printSchema()

In [ ]:
titanic.describe()

In [ ]:
titanic.describe().toPandas()

In [ ]:
### GroupBy 
grouped = titanic.groupby("Sex")

### Grouped Average
grouped.avg().toPandas()

In [ ]:
grouped.count().toPandas()

In [ ]:
### Aggregate multiple columns with different operations 
agg_dict={"Survived":"mean", "Fare":"sum"}
grouped.agg(agg_dict).show() #toPandas()

In [ ]:
### SELECT Columns 
from pyspark.sql import functions as F 

### select missing values
missing_values_cabin = titanic.select(F.sum(F.col("Cabin").isNull().cast("int")))
missing_values_cabin.show()

In [ ]:
### Alias 
titanic.select(F.sum("Fare").alias("Total Fare")).show()

In [ ]:
### Remove null values 
titanic.where((~F.col("Cabin").isNull()) & (~F.col("Embarked").isNull())).toPandas()

In [ ]:
titanic.where((F.col("Cabin").isNull())).toPandas()

**On n'a rien fait car on n'a jamais enregistré le résultat !**

**Exercice:**

Reprendre les opérations de façon à sauvegarder ce dataset modifié.


In [ ]:
!hdfs dfs -rm /stagiaire/XXXX/titanic-clean/*
!hdfs dfs -rmdir /stagiaire/XXXX/titanic-clean

In [ ]:
titanic.write.csv("/stagiaire/XXXX/titanic-clean")

In [ ]:
!hdfs dfs -ls /stagiaire/XXXX/titanic-clean

In [ ]:
titanic2 = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load("/stagiaire/XXXX/titanic-clean/*.csv"))

In [ ]:
titanic2.head(3)

In [ ]:
sc.stop()